KAGGLE ile hibrit model şeklinde gidilmektedir. Veri işlemesi, hazırlığı lokalde yapılırken; Model eğitimi kaggle üzerinden yapılmaktadır.

Bu dosya kaggle da eğitilen modelin kodlarını ve gelişimini arşivlemek açısından birebir kod ve açıklamalarıdır. (Çıktıları Kendiniz Çalıştırarak Görebilir yada Kaggle Hesabıma Gidebilirsiniz)

In [ ]:
""" VERİ YÜKLEME """

import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

# model için hazırladığımız train.data verisi seti
df = pd.read_csv("/kaggle/input/instacart-features-local/train_data.csv")

print(f"✅ Veri yüklendi! Boyut: {df.shape}")
df.head()

Create_Dataset dosyası ile son hale getirdiğimiz eğitim için veri setini sisteme yükledik ve burada çekiyoruz.

In [ ]:
""" DEĞİŞKENLERİN HAZIRLANMASI """

X = df.drop(['user_id', 'product_id', 'reordered'], axis=1) # değişkenler
y = df['reordered'] # hedef

# %80 Eğitim, %20 test
print("🔪 Veri setleri ayrılıyor...")
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Eğitim Seti: {X_train.shape}")
print(f"Test Seti:   {X_val.shape}")

Modelin kullanıcı, ürün ve tahmin etmesi gereken (hedef değişken) sütunlarını görmemesi için kaldırıyoruz diğerlerini alacak. eğer kaldırmaz isek model bunları baz alarak öğrenmek yerine ezber yapar. Yani ahmet şunu almış, muz bu kadar alınıyor gibi ezberden kaçınsın. Eğer ürün popülerse ve kullanıcı daha önce çok aldıysa cevap Evet' yani tekrar alır demesi lazım bu mantıkla yaklaşmalı.


In [ ]:
""" XGBOOST MODELİNİN AYARLARI """

model = xgb.XGBClassifier(
    objective='binary:logistic', # İkili sınıflandırmadır. (binary ikili durumu ifade eder evet-hayır,alacak-almıycak gibi, logistic ise sınıflandırma için kullanırız direkt 0,1 vermesin küsüratlıda versin)
    eval_metric='logloss', # puan sistemi (Çok eminse ve yanlış çıkarsa büyük ceza gibi tahminle orantılı ceza)
    n_estimators=1000,      # Ağaç sayısı
    learning_rate=0.05,     # Öğrenme hızı
    max_depth=6,            # Ağaç derinliği
    tree_method='hist',     # Hızlandırma için (histogram ile veriyi gruplayarak işlem yapsın. hız katar.)
    device='cuda',          # GPU kullanımı (Eğer GPU açtıysan 'cuda', açmadıysan 'cpu' yap)
    early_stopping_rounds=50 # Eğer 50 tur boyunca iyileşme olmazsa dur
)

print("🚀 Model eğitimi başlıyor...")
model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)], # her ağaçtan sonra görmediği verilerle test yapra
    verbose=100  # Her 100 adımda(ağaçta) bir durum raporu ver
)

In [ ]:
""" TAHMİN """

# Tahmin yap
y_pred = model.predict(X_val) # x_val verileri ile tahmin ettirme
# 0 ve 1 leden oluşan tahmin sonuçları y_pred de

# Raporla
print("📊 Model Performansı:")
print(classification_report(y_val, y_pred)) # y_val gerçek cevaplar ile y_pred tahminlerin karşılaştırılması

# Hangi özellik en önemlisi (Interpretabilitiy)
xgb.plot_importance(model, max_num_features=10)  # karar verirken en çok kullandığı özellikleri sıralar
plt.title("Hangi Özellik Daha Önemli?")
plt.show()

predict() metodunda varsayılan olarak eşik değer %50 dir. Yani 1 (alır) sonucunu vermesi için %50.01 ikna olması yeterlidir.

classification_report() metodu ceapların kontrolünü yapar. Direkt şu kadar bildi (accuracy) ölçmez genel bir test yapar. 

Testte Precision (Kesinlik): "Alacak" dediklerinin yüzde kaçı gerçekten aldı

Recall (Duyarlılık - En Önemlisi): Gerçekten alanların yüzde kaçını yakalayabildi

F1-Score: Precision ve Recall'un ortalaması. Modelin genel dengesi. (direkt bu metrik çok şey ifade eder)

Support: o sınıftan (0 veya 1) test setinde kaç tane örnek olduğu.

Interpretability kısmı analiz için çok önemlidir. İlk sonuçlar genelde hiç iç açıcı olmayakcaktır. Bunun tedavisinide en kısa ve kesin yoldan bu adım ile saptayabilir. Modele bu kararları verirken hangi özellikleri kullandın diye sorar. Bu da model gerçekten bizim istediğimiz mantık da düşünüp düşünmediğini gösterir ve nokta atışı tedavi yapabiliriz.

In [ ]:
""" Karmaşıklık Matrisi incelemesi ve En iyi Eşik Değerini Bulma """
import numpy as np
from sklearn.metrics import f1_score, confusion_matrix
import seaborn as sns

# 1-Tahminlerin ihtimal oranları (yani karar verirken ne kadar oranlar ile verdin)
print("Olasılıklar hesaplanıyor")
y_pred_probs = model.predict_proba(X_val)[:, 1]


# 2-En iyi eşik değerini (Threshold) bulma döngüsü
best_threshold = 0
best_f1 = 0
results = []

print("En iyi eşik değeri aranıyor")

# 0.10'dan 0.50'ye kadar 0.02 adım adım dene
for threshold in np.arange(0.1, 0.51, 0.02):
    # Olasılık > threshold ise 1 yap, değilse 0
    preds_custom = (y_pred_probs > threshold).astype(int) # karşılaştırma
    
    # F1 Skorunu hesapla (Class 1 için)
    f1 = f1_score(y_val, preds_custom) # her eşiğe göre hesaplamadığımız sınıf ile gerçek değerler f1 skorunu hesapla
    results.append((threshold, f1)) # bu ölçümleri karşılaştırmak üzere listeye atıyoruz
    
    print(f"   Eşik: {threshold:.2f} -> F1 Score: {f1:.4f}") # her döngü sonucu
    
    # en iyi f1 skoru seçimi
    if f1 > best_f1:
        best_f1 = f1
        best_threshold = threshold

print("-" * 30)
print(f"EN İYİ EŞİK DEĞERİ: {best_threshold:.2f}")
print(f"Yeni F1 Skoru: {best_f1:.4f}")


# 3-En iyi eşik ile DETAYLI raporu bas
final_preds = (y_pred_probs > best_threshold).astype(int) # en iyi eşik değeri al

print("\nOptimize Edilmiş Rapor:")
print(classification_report(y_val, final_preds)) # alınan değerler tekrardan report metodu ile raporla


# 4-Karışıklık Matrisi (Confusion Matrix) - Neyi ne kadar doğru bildik görmek için
cm = confusion_matrix(y_val, final_preds) # cevaplar ile bizim en iyi f1 li tahmnilerimiz
plt.figure(figsize=(6, 4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title(f'Confusion Matrix (Threshold: {best_threshold:.2f})')
plt.ylabel('Gerçek Değer')
plt.xlabel('Tahmin')
plt.show()

predict_proba() ile bu sefer ihtimalleri alıyoruz. yani sen sınıflandırma (0 alma, 1 al) yaparken %50 üstü olunca 1 diyosun ama  ne kadar oran verdin her veri için diye soruyoruz. geriye her tahmini için [[0.3, 0.7], [0.8, 0.2], ...] gibi bir çıktı döner biz de sadece sağdaki yani 1.indeksteki 1(alır) için olan oranları alıyoruz.

pred default olarak threshold değerini %50 alır yani 50.1 olduğu an 1 verir. Burada çok korkak davrandığı tespit ettik çıkan sonuçlarda. Bu değerin en optimum sonuçları ne kadarken verdiğini görmek için tüm değeleri ufak adımlarla değiştire değiştire deniyoruz. ihtimalleri aldığımız kısım ile sürekli denediğimiz eşik değeri karşılaştırıp deniyoruz. En iyi sonucu veren f1 skorlu değerli report fonkisyonu ile detaylı raporluyoruz

Son olarak karışıklık matrisi ile röntgen çekiyoruz. Bu matriste satırlar Gerçek Durumu, sütunlar ise Modelin Tahminini gösterir. Her kutunun ise ayrı bir anlamı vardır. 

Sol üst kutu (TN-TRUE NEGATİVE) Buraya sessiz doğru deneriz. burada müşterinin almayacağı ürüne modelinde almayacak demesidir. saten bu durum her problemde genelde doğrudur. Çünkü bu durumda bulunan bi ton veri saten hep vardır.

Sağ üst kutu (FP-FALSE POSITIVE) Yanlış alarm verdiğimiz durumlar. Müşterinin almayacağı ürüne modelin alacak demesidir. Bu oran son yaptığımız eşik değeri 50 den 20 ye düşürme işlemimizden sonra ciddi arttı. Çünkü model çok daha sık alacak dediği için hata oranıda arttı. Bu sorun bu problem için göz ardı edilebilir çünkü müşteriye ürün önerme maliyeti düşüktür ama bu önermeye karşılık vermesi önemli olduğundan maliyet-kazanç oranımız yüksektir bundan dolayı burada ki hata payı göz ardı edilebilir. (biz az ihtimalde olsa önerelim maliyeti yok ya alırsa gibi)

Sol alt kutu (FN-FALSE NEGATIVE) Bu kısım farkı yaratacağımız, problemin ana amaç kısmıdır. Müşterinin aldığı ancak biz almayacak diyip de önermede bulunmadığımız kısımdır. Eşik değeri azaltmamız burada da ciddi fayda sağladı ancak halen sonuçlara göre yüksek.

Sağ alt kutu (TP-TRUE POSİTİVE) Burası doğru atış yaptığımız yerler. Müşterinin aldığı modelinde alacak dediği kısımdır.

In [ ]:
""" Bayesian Optimizasyonu ile Hiperparametre Ayarı """

import optuna # Bayesian optimizasyonu için kullanılan popüler kütüphane


"""
    Optuna'nın her denemede (trial) çalıştıracağı fonksiyon
    Amacı: Rastgele parametreler seçip, modeli eğitip, başarı puanını döndürmektir.
"""
def objective(trial):
    
    # 1-Denenecek Parametre Uzayı (Arama Alanı)
    params = {
        # Temel parametreler bunlar fixtir değişmeyecekler
        'objective': 'binary:logistic', # İkili sınıflandırma (0-1) yine
        'tree_method': 'hist', # Hızlandırma için histogram tabanlı yöntem
        'device': 'cuda',  # GPU kullanımı
        'eval_metric': 'logloss', # Hata ölçüsü (Log Loss) - Düşürmeye çalışacağız
        
        # Bu parametrelerde ise Verdiğimiz aralıklardan sayısal değerler seçerek deniyecek
        'n_estimators': trial.suggest_int('n_estimators', 500, 1500), # Ağaç sayısı
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1), # Öğrenme hızı
        'max_depth': trial.suggest_int('max_depth', 4, 10), # Ağaç derinliği

        # Regularization (Düzenleme) Parametreleri - en başta Aşırı öğrenmeyi engellemeye yardımcı olur ve modelin genelleme yeteneğini artırır
        'subsample': trial.suggest_float('subsample', 0.6, 1.0), # Her ağaçta verinin % kaçını kullansın?
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0), # Her ağaçta özelliklerin % kaçını kullansın?
        'gamma': trial.suggest_float('gamma', 0, 5) # Ağacın büyümesini kontrol eden fren mekanizması
    }
    
    # 2-Modeli Kur (Seçilen parametrelerle)
    model = xgb.XGBClassifier(**params, early_stopping_rounds=50) # seçilen parametreler ile model kuracak ve erken durdurma mekanizması ekleyecek  
    # (50 tur boyunca iyileşme olmazsa dur demek)
    
    # 3-Eğitim
    # Verbose=False yapıyoruz ki ekranı 20 kere eğitim loglarıyla doldurmasın. Aşağıda kombinasyonu 20 yaptığımızdan.
    model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
    
    # 4-Tahmin Yap (Validasyon Setinde)
    # Burada parametreler kısmında verdiğimiz Log Loss (Hata) değerini minimize etmeye çalışacağız.
    # Çünkü Log Loss ne kadar düşükse, model o kadar "kendinden emin" demektir.
    y_probs = model.predict_proba(X_val)[:, 1] # tine 1 sınıfına ait "alır" olasılıklarını alıyoruz (0-1 arasında değerler)
    loss = log_loss(y_val, y_probs) # gerçek değerler ile tahmin edilen olasılıkların log loss değerini hesapla
    # yani ceza değerleridir - örneğin kesin %99 alacak dedi ama almadı çok ceza olur yani değer yüksek, ama % 50 dedi ve almadı daha az ceza olur yani değer düşük olur
    
    return loss

print("Optuna ile en iyi parametreler aranıyor")

# direction='minimize' -> Çünkü Log Loss (Hata) değeri üzerinden gidiyoruz, bu yüzden minimize etmeye çalışacağız.
# eper yukarıda loss değerleri yerine accuracy gibi bir başarı metriği üzerinden gidiyor olsaydık direction='maximize' yapardık. yani doğruluğu yükseltmeye uğraşsın diye
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20) # 20 farklı kombinasyon dene
# kombinasyon her değiştiğinde objective fonksiyonu çalışacak, yeni parametreler seçecek, modeli eğitecek ve log loss değerini döndürecek duruma göre değişimler yapıp tekrar çağırıp ölçecek

print("BULUNAN EN İYİ PARAMETRELER:")
print(study.best_params)
print(f"En Düşük Hata (LogLoss): {study.best_value}")

BULUNAN EN İYİ PARAMETRELER:
{'n_estimators': 938, 'learning_rate': 0.03743533184090101, 'max_depth': 10, 'subsample': 0.6605204427574461, 'colsample_bytree': 0.7019846992999667, 'gamma': 1.54649974715052}
En Düşük Hata (LogLoss): 0.2632623644411835

Bu sonuçlar en iyi parametlerdir. Bunlar ve tekrar bu yeni model ile eşik değeri ayarı yapınca yeni modelimize kavuşacağız

In [ ]:
""" Yeni Parametrelerle Model Eğitimi """

# yukarda incelenmiş aynı kodlar ile;
# Optuna'dan gelen Kazanan Parametreler
final_params = {
    'n_estimators': 938,
    'learning_rate': 0.03743533184090101,
    'max_depth': 10,
    'subsample': 0.6605204427574461,
    'colsample_bytree': 0.7019846992999667,
    'gamma': 1.54649974715052,
    
    # Sabit Ayarlar
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'tree_method': 'hist',
    'device': 'cuda'
}

print("Model Eğitimi (Optimize Parametrelerle)")
final_model = xgb.XGBClassifier(**final_params, early_stopping_rounds=50)
final_model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    verbose=100
)

# Yeni Eşik Değeri (Threshold) Bulma
# Model değiştiği için eski eşik (0.20) artık geçerli olmayabilir.
print("\n🔍 Yeni model için en iyi eşik değeri aranıyor...")
y_pred_probs = final_model.predict_proba(X_val)[:, 1]
best_threshold = 0
best_f1 = 0
for threshold in np.arange(0.1, 0.51, 0.01): # Daha hassas arama (0.01 adımlarla)
    preds_custom = (y_pred_probs > threshold).astype(int)
    f1 = f1_score(y_val, preds_custom)
    if f1 > best_f1:
        best_f1 = f1
        best_threshold = threshold

print(f"YENİ KAZANAN EŞİK: {best_threshold:.2f}")
print(f"YENİ F1 SKORU: {best_f1:.4f}")

final_preds = (y_pred_probs > best_threshold).astype(int)
print("\nOptimize Edilmiş Final Rapor:")
print(classification_report(y_val, final_preds))

final_model.save_model("xgb_instacart_optimized_v2.json")
print("\nModel kaydedildi: xgb_instacart_optimized_v2.json")

Çıkan sonuçlara göre yine aynı eşik değer 0.20 ve f1 skoruda 0.3932 yani ayar yapılmamış modelden binde 2 daha düşük yani aynı. Diğer değerlerde birebir aynı buradan çıkaracağımız şey EDA kısmından alabileceğimiz her şeyi almışız model saten tüm veridğimiz özellikleri yutmuş daha fazla birşey çıkaramam diyor bundan dolayı future kısmına dönüp orada geliştirme yapacağız.

In [ ]:
""" Yeni Özelliklerle (v2) Model Eğitimi """

# yeni v2 model özellikleri eklediğimden eğitim için gönderdiğimiz veri setini de değiştirmemiz gerekiyor. 
# O yüzden tekrar veri yükleme kısmına yenisini gönderiyoruz - totalde 8.5M satır
DATA_PATH = "/kaggle/input/train-v2/train_data.csv"
print("Veri yükleniyor")
df = pd.read_csv(DATA_PATH)

# Eksik veri kontrolü (Yok hallettikde garanti olsun null olmasın eksik varsa doldurma)
df = df.fillna(0)

# Özellikleri ve Hedefi Ayır
# Artık X'in içinde yeni eklediğimiz 'orders_since_last_bought' vb. hepsi var.
# bu üçünü çıkarmamız yetiyor kalan hepsini x olarak alacak
X = df.drop(['user_id', 'product_id', 'reordered'], axis=1)
y = df['reordered']

print(f"Veri Hazır, Özellik Sayısı: {X.shape[1]}")
print("Özellik Listesi:", list(X.columns)) # kaçırdığımız olmasın gözden geçirelim diye özelliklerin listesini de basıyoruz

# Eğitim ve Test Setine Böl (%80 - %20)
print("\n Veri bölünüyor")
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Optuna İle Bulduğumuz en iyi Parametreler
params = {
    'n_estimators': 938,
    'learning_rate': 0.03743533184090101,
    'max_depth': 10,
    'subsample': 0.6605204427574461,
    'colsample_bytree': 0.7019846992999667,
    'gamma': 1.54649974715052,
    
    # Sabitler
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'tree_method': 'hist',
    'device': 'cuda' # GPU yoksa 'cpu' yap yoksa hızlanmaz
}

# Modeli Eğit
print("\n Final Model Eğitimi Başlıyor")
model = xgb.XGBClassifier(**params, early_stopping_rounds=50)
model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    verbose=100 # Her 100 ağaçta bir rapor verecek loss değerini filan arada görmemiz nasıl gittiğini anlamamızı sağlar
)

# Yine garanti olsun bu setinde En İyi Eşik Değerini (Threshold) Bulalım
print("\n En iyi eşik değeri aranıyor")
y_pred_probs = model.predict_proba(X_val)[:, 1]

best_threshold = 0.20 # Başlangıç (önceki en iyiydi)
best_f1 = 0

for threshold in np.arange(0.15, 0.45, 0.01):
    preds = (y_pred_probs > threshold).astype(int)
    score = f1_score(y_val, preds)
    if score > best_f1:
        best_f1 = score
        best_threshold = threshold

print(f"KAZANAN EŞİK: {best_threshold:.2f}")
print(f"YENİ F1 SKORU: {best_f1:.4f}")

# Detaylı Rapor
final_preds = (y_pred_probs > best_threshold).astype(int)
print("\n Final Karne:")
print(classification_report(y_val, final_preds))

# Hangi Özellik İşe Yaradı (Feature Importance) diğerlerine göre göreceli olarak hangi özellikler daha etkili olmuş karar verirken
plt.figure(figsize=(10, 6))
xgb.plot_importance(model, max_num_features=15, importance_type='weight', title='Hangi Özellik En Çok İşe Yaradı')
plt.show()

# Modeli Kaydet
model.save_model("xgb_final_v3.json")
print("\n Model Kaydedildi: xgb_final_v3.json")

öncelikle yeni eşik değeri 0.22 oldu. f1 skoru 0.43 hatasıda 0.24 geldi. bu oranlar önceki modele göre çok ciddi daha iyi. üstelik recall (yani balığı tutma diyim) değeri %49 a ulaştı yani her iki önermemizden birinde müşteriyi yakalıyoruz. Süpriz ise en çok kullandığı özelliklerde oldu. user_avg_basket_size özelliğini en çok kullanmış yani artık diğer model gibi en popüler hangisi ise reorder_Ratio değerine (şimdiki modelde bu değer en altta) göre değilde gerçekten müşteriyi analiz ederek kararlar veriyor.

In [ ]:
""" Karışıklık Matrisi """
# Tahminleri Eşik Değerine Göre Yap (0.22 bulmuştuk)
# matris için alır-almaz tahminlerini yapalım. 
# çünkü matris probs dan çıkan sayısal değerli örneğin 0.88 alır gibi ifadelerden anlamaz onun yerine 0-1 alır-almaz kesin kodlarla yaparız
threshold = best_threshold 
preds = (y_pred_probs > threshold).astype(int)

# Matrisi Hesapla
cm = confusion_matrix(y_val, preds)

# Görselleştir (Heatmap)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Tahmin: Almaz (0)', 'Tahmin: Alır (1)'],
            yticklabels=['Gerçek: Almadı (0)', 'Gerçek: Aldı (1)'])
plt.ylabel('Gerçek Durum')
plt.xlabel('Modelin Tahmini')
plt.title(f'Final Model Karışıklık Matrisi (Eşik: {threshold:.2f})')
plt.show()

TN - Saten çoğu belli olan durumlar. önemsizdir. müşterinin almadığı modelinde almayacak dediği

FP - Model Alacak dedi ama müşteri almadı. Bu kötü bir şey değil aslında. E-ticaret sitelerinde Bunları da sevebilirsiniz kısmında çıkan ama tıklamadığın ürünlerdir. Müşteriyi çok rahatsız etmez, sadece bir öneridir.

FN - Müşteri ürünü aldı ama model Almaz sanıyordu burası kaçan fırsattır. önceki modelden yaklaşık %5 daha iyi durumda. Bu sektör için bu değer gayet iyiymiş. asıl odaklanmamız gereken kısımdır.

TP - Burası doğru bildiğimiz müşterinin aldığı, modelinde alacak dediği yerdir. yani direkt kasaya giren paradır.